In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense

**Directory Path**

In [ ]:
Training_path = "dataset/seg_train"
Validation_path = "dataset/seg_test"

In [ ]:
# Load and preprocess training images
labels_train = []
images_train = []
# Iterate over subdirectories in Training_path
for label in os.listdir(Training_path):
    #print("打印label:",label)
    imgpath = os.path.join(Training_path, label)
    #  print("打印imgpath:",imgpath)

    # Check if it's a directory
    if os.path.isdir(imgpath):
       #   print("走进")
        # Iterate over files in the directory
        for file in os.listdir(imgpath):
            imagess = os.path.join(imgpath, file)
           #   print("打印imagess:",imagess)
            # Check if it's a file
            if os.path.isfile(imagess):
               # print("打印labels_train追加")
                # print("打印labels_train追加:",label)
                # print("打印labels_train追加imgpath:",imgpath)
                labels_train.append(label)
                images_train.append(imagess)
            else:
                print(f"Skipping non-file: {imagess}")
    else:
        print(f"Skipping non-directory: {imgpath}")

# Create a dictionary to map labels to numerical values
label_to_num = {label: i for i, label in enumerate(np.unique(labels_train))}
#print("打印labels_train的值1111:",labels_train)
for label in enumerate(np.unique(labels_train)):
    print("打印label2222222:",label)

num_labels = len(label_to_num)

# Convert training labels to numerical values
labels_train = [label_to_num[label] for label in labels_train]

# Print some diagnostic information
print("Number of labels:", len(labels_train))
print("Unique labels:", np.unique(labels_train))
# print("打印imgpath:",imgpath)
# print("打印label33333:",label)

# Check if labels_train is empty
if not labels_train:
    raise ValueError("Empty labels_train array")


# Load and preprocess training images
X_train = []
for img_path in images_train:
    try:
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (150, 150))
            img = img / 255.0  # Normalize pixel values to [0, 1]
            X_train.append(img)
        else:
            print(f"Error reading image: {img_path}")
    except Exception as e:
        print(f"Error processing image: {img_path}, {e}")

X_train = np.array(X_train)

# Print some diagnostic information
print("X_train shape:", X_train.shape)

# Convert labels to categorical
Y_train = to_categorical(labels_train, num_labels)

# Load and preprocess validation images
labels_val = []
images_val = []

# Iterate over subdirectories in Validation_path
for label in os.listdir(Validation_path):
    imgpath = os.path.join(Validation_path, label)

    # Check if it's a directory
    if os.path.isdir(imgpath):
        # Iterate over files in the directory
        for file in os.listdir(imgpath):
            imagess = os.path.join(imgpath, file)

            # Check if it's a file
            if os.path.isfile(imagess):
                labels_val.append(label)
                images_val.append(imagess)
            else:
                print(f"Skipping non-file: {imagess}")
    else:
        print(f"Skipping non-directory: {imgpath}")

# Convert validation labels to numerical values
labels_val = [label_to_num[label] for label in labels_val]

# Load and preprocess validation images
X_val = []
for img_path in images_val:
    try:
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (150, 150))
            img = img / 255.0  # Normalize pixel values to [0, 1]
            X_val.append(img)
        else:
            print(f"Error reading image: {img_path}")
    except Exception as e:
        print(f"Error processing image: {img_path}, {e}")

X_val = np.array(X_val)

# Print some diagnostic information
#print("X_val shape:", X_val.shape)

# Convert labels to categorical
Y_val = to_categorical(labels_val, num_labels)
#print("打印labels_val:",labels_val)
#print("打印num_labels:",num_labels)
#print("打印Y_val:",Y_val)

**Building CNN Model**

In [ ]:
# Define the CNN model

input_layer = Input(shape=(150, 150, 3))

# Convolutional layers
conv1 = Conv2D(16, (3, 3), activation='relu')(input_layer)
maxpool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(32, (3, 3), activation='relu')(maxpool1)
maxpool2 = MaxPooling2D((2, 2))(conv2)

# Flatten and fully connected layers
flatten = Flatten()(maxpool2)
dense1 = Dense(128, activation='relu')(flatten)
output_layer = Dense(2, activation='softmax')(dense1)  ###要改
model = Model(inputs=input_layer, outputs=output_layer)

# Display model summary
model.summary()


**Compile Model ,Training & Evaluating**

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation to increase model generalization
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)

# Train the model with validation data
batch_size = 32
epochs = 100 
model.fit(datagen.flow(X_train, Y_train, batch_size=batch_size), epochs=epochs, validation_data=(X_val, Y_val))

# Evaluate the model on training data
train_loss, train_acc = model.evaluate(X_train, Y_train)
print(f'Training Accuracy: {train_acc}')

# Evaluate the model on validation data
val_loss, val_acc = model.evaluate(X_val, Y_val)
print(f'Validation Accuracy: {val_acc}')

# Save the model


In [ ]:
model.save('Image_Classification_Model_03_1000.h5')

In [ ]:
print(f'Training Accuracy: {train_acc}')
print(f'Validation Accuracy: {val_acc}')

**Draw a Confusion Matrix**

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import dtale


import datetime
# 获取当前时间
current_time = datetime.datetime.now()
# 将时间格式化为字符串
formatted_time = current_time.strftime("%Y%m%d%H%M%S")

path="dataset/seg_pred/"



for  root, dirs, files in os.walk(path):
    result_data=[]
    column_name_data=["picture_name",'predict_label'] 
    
    for file in files:
        X_val = []
        column_name =[] 
        data1=[]
        img = cv2.imread(path+file)
        if img is not None:
            img = cv2.resize(img, (150, 150))
            img = img / 255.0  # Normalize pixel values to [0, 1]
            X_val.append(img)
        X_val = np.array(X_val)

        from keras.models import load_model
        model = load_model('Image_Classification_Model_03_1000.h5')

        prediction = model.predict(X_val)

        #labels={0:'cemian',1:'ceph',2:"inner_bottom",3:"inner_center",4:"inner_left",5:"inner_right",6:"inner_top",7:"weixiao",8:"zhengmian"}
        labels={0:'ceph',1:'other'}
       # column_name_data.append(file)
        data1.append(file)
        idx=0
        for j in range(5):
            predicted_label = labels[np.argmax(prediction[idx])]
        data1.append(predicted_label)
        # print("打印data1的数据:",data1)
        result_data.append(data1)
    # print("打印column_name_data:",column_name_data)
    # print("打印result_data:",result_data)
   # result_data=["weixiao34.JPG","zheng"]
    df = pd.DataFrame(result_data, columns=column_name_data)
    df.to_excel('图片分类_'+formatted_time+'.xlsx', index=True)
    dtale.show(df, open_browser=True)

2023-12-19 19:21:31.177275: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-19 19:21:31.178399: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 19:21:31.198437: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 19:21:31.199003: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-19 19:21:31.475264: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

1/1 [==============================] - 0s 109ms/step


NameError: name 'idx' is not defined